# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `pub_paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [41]:
# !cat publications.tsv
!type publications.tsv

pub_date	status	status2	title	venue	excerpt	abstract	coauthors	citation	cv_citation_authors	cv_citation_other	url_slug	pub_paper_url	wp_paper_url	slides_url	opt0_text	opt0_url	opt1_text	opt1_url	opt2_text	opt2_url
11/1/2023	work_in_progress		What Do We Learn From Reading Every FOMC Transcript?				"Olivier Coibion, Yuriy Gorodnichenko and Cooper Howes"	"Coibion, Olivier, Marc Dordal i Carreras, Yuriy Gorodnichenko, and Cooper Howes. ""What Do We Learn From Reading Every FOMC Transcript?""  <i>In Progress</i> (2023)."	"Coibion, Olivier, Marc Dordal i Carreras, Yuriy Gorodnichenko, and Cooper Howes."	<i>In Progress</i> (2023).	CDGH_FOMC_transcript									
11/1/2023	work_in_progress		"Gender Gap, Structural Change and Female Comparative Advantage: A Quantitative Analysis of China"		"This paper presents a theoretical framework to reconcile the declining female labor force participation (FLFP) rate and the diverging gender gap in workforce participation in China with the expansion of the se

## Import pandas

We are using the very handy pandas library for dataframes.

In [44]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [47]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,status,status2,title,venue,excerpt,abstract,coauthors,citation,cv_citation_authors,...,url_slug,pub_paper_url,wp_paper_url,slides_url,opt0_text,opt0_url,opt1_text,opt1_url,opt2_text,opt2_url
0,11/1/2023,work_in_progress,NaN,What Do We Learn From Reading Every FOMC Trans...,NaN,NaN,NaN,"Olivier Coibion, Yuriy Gorodnichenko and Coope...","Coibion, Olivier, Marc Dordal i Carreras, Yuri...","Coibion, Olivier, Marc Dordal i Carreras, Yuri...",...,CDGH_FOMC_transcript,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11/1/2023,work_in_progress,NaN,"Gender Gap, Structural Change and Female Compa...",NaN,This paper presents a theoretical framework to...,This paper presents a theoretical framework to...,Cassie Xiang,"Dordal i Carreras, Marc and Cassie Xiang. ""Gen...","Dordal i Carreras, Marc and Cassie Xiang.",...,DX_Gender_Gap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/1/2023,work_in_progress,NaN,A New Indeterminacy with Fluctuations in Volat...,NaN,NaN,NaN,Seung Joo Lee,"Dordal i Carreras, Marc and Seung Joo Lee. ""A ...","Dordal i Carreras, Marc and Seung Joo Lee.",...,DJ_Indeterminacy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/26/2024,working_paper,submitted,Consumer Perceptions and Willingness to Adopt ...,NaN,This study explores public perceptions of reta...,This study investigates the public's perceptio...,"Kohei Kawaguchi, Si Yuan Jin and Haicheng Guo","Dordal i Carreras, Marc, Kohei Kawaguchi, Si Y...","Dordal i Carreras, Marc, Kohei Kawaguchi, Si Y...",...,DKJG_eHKD_pilot,NaN,http://marcdordal.github.io/files/WP_rCBDC_Tas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12/26/2024,working_paper,submitted,The Spatial Transmission of US Banking Panics:...,NaN,We study the propagation of localized banking ...,We study the propagation of localized banking ...,Seung Joo Lee,"Dordal i Carreras, Marc and Seung Joo Lee. ""Th...","Dordal i Carreras, Marc and Seung Joo Lee.",...,DJ_Bank_Panics,NaN,http://marcdordal.github.io/files/WP_US_Panics...,http://marcdordal.github.io/files/slides_US_Pa...,NaN,NaN,NaN,NaN,NaN,NaN
5,8/1/2024,working_paper,submitted,Higher-Order Forward Guidance,NaN,This paper presents a model of the business cy...,This paper presents a model of the business cy...,Seung Joo Lee,"Dordal i Carreras, Marc and Seung Joo Lee. ""Hi...","Dordal i Carreras, Marc and Seung Joo Lee.",...,DJ_Higher_OrderFW,NaN,http://marcdordal.github.io/files/WP_Higher_or...,http://marcdordal.github.io/files/slides_Highe...,NaN,NaN,NaN,NaN,NaN,NaN
6,8/1/2024,working_paper,submitted,Endogenous Firm Entry and the Supply-Side Effe...,NaN,We present a model of business cycles with end...,We present a model of business cycles with end...,Seung Joo Lee and Zhenghua Qi,"Dordal i Carreras, Marc, Seung Joo Lee, and Zh...","Dordal i Carreras, Marc, Seung Joo Lee, and Zh...",...,DJQ_ASAD_revisited,NaN,http://marcdordal.github.io/files/WP_Firm_Entr...,http://marcdordal.github.io/files/slides_Firm_...,NaN,NaN,NaN,NaN,NaN,NaN
7,8/1/2024,working_paper,submitted,A Unified Theory of the Term-Structure and Mon...,NaN,We develop a New-Keynesian framework that inco...,We develop a New-Keynesian framework that inco...,Seung Joo Lee,"Joo Lee, Seung, and Marc Dordal i Carreras. ""A...","Joo Lee, Seung, and Marc Dordal i Carreras.",...,JD_term_structure_theory,NaN,http://marcdordal.github.io/files/WP_Term_Stru...,http://marcdordal.github.io/files/slides_Term_...,NaN,NaN,NaN,NaN,NaN,NaN
8,8/1/2024,working_paper,submitted,Self-fulfilling Volatility and a New Monetary ...,NaN,We demonstrate that macroeconomic models with ...,We demonstrate that macroeconomic models with ...,Seung Joo Lee,"Joo Lee, Seung, and Marc Dordal i Carreras. ""S...","Joo Lee, Seung, and Marc Dordal i Carreras.",...,JD_sunspot,NaN,http://marcdordal.github.io/files/WP_Sunspot_r...,http://marcdordal.github.io/files/slides_Sunsp...,NaN,NaN,NaN,NaN,NaN,NaN
9,8/1/2024,working_paper,submitted,"Efficiency, Risk and the Gains from Trade in I...",NaN,We propose a model of the financial sector tha...,We propose a model of the

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [50]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Delete existing markdown files

In [53]:
import os
import shutil

# Replace these with the actual folder paths you want to clear.
base_path = 'C:/Users/marcd_ncckl9u/Documents/GitHub/marcdordal.github.io'
folder_names = ['_published_papers', '_policy_papers', '_working_papers', '_works_in_progress', '_resting_works']

# Loop through each folder path
for folder in folder_names:
    # Construct folder path
    folder_path = base_path + "/" + folder
    # Check if the folder exists
    if os.path.exists(folder_path):
        # Remove all contents of the folder
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Removes files and links
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Removes subdirectories
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
    else:
        print(f'The folder {folder} does not exist or cannot be found.')


The folder _published_papers does not exist or cannot be found.
The folder _policy_papers does not exist or cannot be found.
The folder _working_papers does not exist or cannot be found.
The folder _works_in_progress does not exist or cannot be found.
The folder _resting_works does not exist or cannot be found.


## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [56]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ndate: " + str(item.pub_date) + '\n'
    
    if item.status == "published_paper":
        md += """collection: published_papers"""
        md += """\npermalink: /published_paper/""" + html_filename
            
    elif item.status == "working_paper":
        md += """collection: working_papers"""
        md += """\npermalink: /working_paper/""" + html_filename
            
    elif item.status == "policy_paper":
        md += """collection: policy_papers"""
        md += """\npermalink: /policy_paper/""" + html_filename
            
    elif item.status == "work_in_progress":
        md += """collection: works_in_progress"""
        md += """\npermalink: /work_in_progress/""" + html_filename
            
    elif item.status == "resting_work":
        md += """collection: resting_works"""
        md += """\npermalink: /resting_work/""" + html_filename
    
    if len(str(item.status2)) > 5:
        md += "\nstatus2: '" + html_escape(item.status2) + "'"
    
#     md = "---\ntitle: \""   + item.title + '"\n'

    md += "\ntitle: \""   + item.title + '"'

    if len(str(item.venue)) > 5:
        md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
        md += "\ncustom_excerpt: " + "'true'"
    
    if len(str(item.abstract)) > 5:
        md += "\nabstract: '" + html_escape(item.abstract) + "'"
        
    if len(str(item.coauthors)) > 5:
        md += "\ncoauthors: '" + html_escape(item.coauthors) + "'"
    
    if len(str(item.citation)) > 5:
        md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    if len(str(item.cv_citation_authors)) > 5:
        md += "\ncv_citation_authors: '" + html_escape(item.cv_citation_authors) + "'"
        
    if len(str(item.cv_citation_other)) > 5:
        md += "\ncv_citation_other: '" + html_escape(item.cv_citation_other) + "'"
    
    if len(str(item.url_slug)) > 5:
        md += "\nurl_slug: '" + html_escape(item.url_slug) + "'"
    
    if len(str(item.pub_paper_url)) > 5:
        md += "\npubpaperurl: '" + item.pub_paper_url + "'"
    
    if len(str(item.wp_paper_url)) > 5:
        md += "\nwppaperurl: '" + item.wp_paper_url + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"
     
    for i in range(3):
        opt_url = item[f'opt{i}_url']
        opt_text = item[f'opt{i}_text']
        
        if len(str(opt_url)) > 5:
            md += "\n" f'opt{i}text' ": '" + opt_text + "'"
            md += "\n" f'opt{i}url' ": '" + opt_url + "'"
    
    md += "\n---\n"
    
    ## Markdown description for individual page
        
    if len(str(item.abstract)) > 5:
        md += "Abstract: " + html_escape(item.abstract) + "\n\n"
    
    if len(str(item.pub_paper_url)) > 5:
        md += "[[Publication](" + item.pub_paper_url + ")] " 
    
    if len(str(item.wp_paper_url)) > 5:
        md += "[[Draft](" + item.wp_paper_url + ")] " 

    if len(str(item.slides_url)) > 5:
        md += "[[Slides](" + item.slides_url + ")] " 
    
    for i in range(3):
        opt_url = item[f'opt{i}_url']
        opt_text = item[f'opt{i}_text']
        
        if len(str(opt_url)) > 5:
            md += "\n[[" + opt_text + "](" + opt_url + ")] "     
    
    
    if len(str(item.citation)) > 5:
        if len(str(item.pub_paper_url)) > 5:
            md += "\n\nRecommended citation: " + item.citation + " " + item.pub_paper_url
        elif len(str(item.wp_paper_url)) > 5:
            md += "\n\nRecommended citation: " + item.citation + " " + item.wp_paper_url
        else:
            md += "\n\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
    
    
    ## Save markdown file into corresponding GitHub folder
    if item.status == "published_paper":
        with open("../_published_papers/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "working_paper":
        with open("../_working_papers/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "policy_paper":
        with open("../_policy_papers/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "work_in_progress":
        with open("../_works_in_progress/" + md_filename, 'w') as f:
            f.write(md)
            
    elif item.status == "resting_work":
        with open("../_resting_works/" + md_filename, 'w') as f:
            f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [59]:
!ls ../_working_papers/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [24]:
!cat ../_working_papers/2024-JD_sunspot.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
